<a href="https://colab.research.google.com/github/mamekin05108/signatecup2024summer/blob/main/lightcat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 7.2 MB/s eta 0:00:00


In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
import lightgbm as lgb
from sklearn import metrics
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold

import xgboost as xgb
from xgboost import XGBClassifier


from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier, Pool
from sklearn.isotonic import IsotonicRegression


# warningsを非表示にする
warnings.filterwarnings("ignore")

In [84]:
df_train = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/fdata/df_train_allnum.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/fdata/df_test_allnum.csv")
ss = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/data/sample_submit.csv", header=None)

In [85]:
train_y = df_train[target]
train_x = df_train.drop(target, axis=1)

In [86]:
test_x = df_test
test_x = test_x.drop(target, axis=1)

In [91]:
test_x.head()

,id,Age,TypeofContact,DurationOfPitch,Gender,NumberOfPersonVisiting,NumberOfFollowups,NumberOfTrips,Passport,MonthlyIncome,...,PitchSatisfactionScore_5,Designation_AVP,Designation_Executive,Designation_Manager,Designation_Senior Manager,Designation_VP,MaritalStatus_未婚,MaritalStatus_独身,MaritalStatus_結婚,MaritalStatus_離婚
0,3489,0.938701,1,-0.212100,0,-1.908669,0.361786,2.196148,0,1.863454,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,3490,-0.922728,1,-0.356740,1,-1.908669,0.361786,0.459028,1,-0.742793,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,3491,-1.439792,1,-0.790661,1,-1.908669,0.361786,-1.278092,0,-1.272115,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,3492,-1.853443,0,-1.079941,0,-1.908669,0.361786,-1.278092,0,-1.273769,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,3493,0.214812,0,-1.079941,0,-1.908669,0.361786,-1.278092,0,-1.155267,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


# モデル


In [87]:
params_lgb = {
    "n_estimators": 3000,
    "learning_rate": 0.01,
    "colsample_bytree": 0.8,
    "subsample_freq": 1,
    "subsample": 0.8,
    "random_seed": 0,
}

In [88]:
params_cat = {
    "iterations": 3000,  # n_estimatorsに相当
    "learning_rate": 0.01,
    "colsample_bylevel": 0.8,  # colsample_bytreeに相当
    "random_seed": 0,
    "verbose": 100,  # 学習の進捗を100回毎に表示
    "use_best_model": True,  # 早期停止のための設定
}

In [89]:
params_xgb = {
    "n_estimators": 3000,
    "learning_rate": 0.01,
    "colsample_bytree": 0.8,
    "subsample": 0.8,
    "early_stopping_rounds": 100,
    "random_state": 0,
}

In [90]:
base_models = {
    'LightGBM': lgb.LGBMClassifier(**params_lgb),
    'CatBoost': CatBoostClassifier(**params_cat, early_stopping_rounds=100),
    'XGBoost': XGBClassifier(**params_xgb)
}

In [104]:
#def train_base_models(X, y, models, cols_category, n_splits=5):
def train_base_models(X, y, models, n_splits=5):
    meta_features = np.zeros((len(X), len(models)))
    oof_predictions = np.zeros(len(X))

    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=)

    for i, (model_name, model) in enumerate(models.items()):
        print(f"Training {model_name}...")
        oof_predictions_model = np.zeros(len(X))

        for fold, (trn_idx, val_idx) in enumerate(cv.split(X, y), start=1):
            trn_x, trn_y = X.iloc[trn_idx], y.iloc[trn_idx]
            val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]

            if isinstance(model, lgb.LGBMClassifier):
                model.fit(
                    trn_x, trn_y,
                    eval_set=[(val_x, val_y)],
                    callbacks=[lgb.early_stopping(100, verbose=False)],
                    #categorical_feature=cols_category,
                )
            elif isinstance(model, CatBoostClassifier):
                #train_pool = Pool(data=trn_x, label=trn_y, cat_features=cols_category)
                #val_pool = Pool(data=val_x, label=val_y, cat_features=cols_category)
                train_pool = Pool(data=trn_x, label=trn_y)
                val_pool = Pool(data=val_x, label=val_y)
                model.fit(train_pool, eval_set=val_pool, verbose=False)
            elif isinstance(model, XGBClassifier):
                model.fit(
                    trn_x, trn_y,
                    eval_set=[(val_x, val_y)],
                    verbose=False,
                )
            else:
                model.fit(trn_x, trn_y)

            oof_predictions_model[val_idx] = model.predict_proba(val_x)[:, 1]

            auc = roc_auc_score(val_y, oof_predictions_model[val_idx])
            print(f"  Fold {fold}, AUC: {auc:.4f}")

        meta_features[:, i] = oof_predictions_model
        oof_predictions += oof_predictions_model / len(models)

        auc = roc_auc_score(y, oof_predictions_model)
        print(f"{model_name} OOF AUC: {auc:.4f}")

    return meta_features, oof_predictions

In [105]:
def train_meta_model(meta_features, y, meta_model, n_splits=5):
    oof_predictions = np.zeros(len(y))
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=815)

    for fold, (trn_idx, val_idx) in enumerate(cv.split(meta_features, y), start=1):
        trn_x, trn_y = meta_features[trn_idx], y.iloc[trn_idx]
        val_x, val_y = meta_features[val_idx], y.iloc[val_idx]

        meta_model.fit(trn_x, trn_y)
        oof_predictions[val_idx] = meta_model.predict_proba(val_x)[:, 1]

        auc = roc_auc_score(val_y, oof_predictions[val_idx])
        print(f"Meta Model Fold {fold}, AUC: {auc:.4f}")

    auc = roc_auc_score(y, oof_predictions)
    print(f"Meta Model OOF AUC: {auc:.4f}")

    return oof_predictions

In [106]:
def stacking_predict(X, base_models, meta_model):
    meta_features = np.column_stack([model.predict_proba(X)[:, 1] for _, model in base_models.items()])
    return meta_model.predict_proba(meta_features)[:, 1]

In [107]:
# ベースモデルの学習とメタ特徴量の生成
meta_features, oof_predictions_base = train_base_models(train_x, train_y, base_models)


Training LightGBM...
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 397, number of negative: 2394
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000380 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 683
[LightGBM] [Info] Number of data points in the train set: 2791, number of used features: 41
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.142243 -> initscore=-1.796785
[LightGBM] [Info] Start training from score -1.796785
  Fold 1, AUC: 0.8216
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 397, number of negative: 2394
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000373 s

In [96]:
meta_features

array([[0.74960532, 0.80590773, 0.76238215],
       [0.13154333, 0.13122678, 0.13168739],
       [0.18607262, 0.20921726, 0.17927241],
       ...,
       [0.04767918, 0.04328016, 0.04015868],
       [0.16035798, 0.20988039, 0.13757175],
       [0.05371685, 0.07459175, 0.07032096]])

In [108]:
meta_features

array([[0.737759  , 0.77190878, 0.73978013],
       [0.12077513, 0.09404334, 0.13856313],
       [0.27930963, 0.25547616, 0.26145357],
       ...,
       [0.04552659, 0.0521905 , 0.04793171],
       [0.19331348, 0.18847833, 0.17051966],
       [0.06632617, 0.08691898, 0.07525136]])

In [124]:
params_logistic = {
    "penalty": 'l2',             # L2正則化
    "C": 1.0,                    # 正則化の強さ
    "solver": 'lbfgs',           # 最適化アルゴリズム
    "max_iter": 100,             # 最大反復回数
    #"class_weight":'balanced',    # クラスの重みづけ
    "random_state": 1506            # 乱数シード
}

In [125]:
# メタモデルの定義と学習
from sklearn.linear_model import LogisticRegression
meta_model = LogisticRegression(**params_logistic)
oof_predictions_meta = train_meta_model(meta_features, train_y, meta_model)


Meta Model Fold 1, AUC: 0.8169
Meta Model Fold 2, AUC: 0.8423
Meta Model Fold 3, AUC: 0.8458
Meta Model Fold 4, AUC: 0.8068
Meta Model Fold 5, AUC: 0.8668
Meta Model OOF AUC: 0.8345


Meta Model Fold 1, AUC: 0.8172
Meta Model Fold 2, AUC: 0.8446
Meta Model Fold 3, AUC: 0.8359
Meta Model Fold 4, AUC: 0.8211
Meta Model Fold 5, AUC: 0.8641
Meta Model OOF AUC: 0.8356

In [126]:
final_auc = roc_auc_score(train_y, oof_predictions_meta)
print(f"Final Stacking Model OOF AUC: {final_auc:.8f}")

Final Stacking Model OOF AUC: 0.83454941


In [128]:
test_predictions = stacking_predict(test_x, base_models, meta_model)

In [132]:
count = 0

In [133]:
from datetime import datetime
import pytz
# カウント変数をインクリメント
count += 1
# 日本時間を取得
japan_tz = pytz.timezone('Asia/Tokyo')
now = datetime.now(japan_tz)
timestamp = now.strftime("%Y%m%d_%H%M%S")

file_name = f"/content/drive/MyDrive/Signate/2024summer/lightcatxgb_en_{timestamp}_{count:03d}.csv"
ss[1] = test_predictions
ss.to_csv(file_name, header=False, index=False)